[Table of Contents](table_of_contents.ipynb)

# Topic 16.  Recursive Least Squares
Author: Seth Nielsen - sethmnielsen@gmail.com
    

##  Introduction
Give a high level (i.e., no equations) explanation of the topic, and why it is important.

In traditional 



* Least squares solution assumes the availability of a set of data (where the prewindowing method is used), and the computations are done on the entire block of data.
* LS is turned into an adaptive algorithm by computing a new update to $h$ (the coefficients minimizing the LS error) as new data become available

## Explanation of the theory

Give a detailed discussion (i.e., equations galore) of the topic.  
The emphasis here is clarity for future students learning the topic.

Least squares:

\begin{equation}
Rh = A^H d, \text{ where } R = A^H A = \sum_{i=1}^N q_{i}q_{i}^H \\
\text{with } q_i = \begin{bmatrix}f[i]\\f[i-1]\\...\\f[i-m+1]\end{bmatrix} \\
\text{Let } p = A^H d = \sum_{i=1}^N q_{i}d_i \\
h = R^{-1}A^{H} = R^{-1}p \\
\end{equation}

Start RLS algorithm:

\begin{equation}
R_t = \sum_{i=1}^t q_{i}q_{i}^H \\
p_t = \sum_{i=1}^t q_{i}d_i = p_{t-1} + q_{t}d_{t} \\
h_t = R_{t}^{-1}p_t \\
\end{equation}

Algorithm is now adaptive, but needs to be recursive ($R_{t}^{-1}$ is currently calculated at each time step). We can break $R_t$ up as

\begin{align}
R_t &= \sum_{i=1}^{t-1}{q_i}{q_{i}^H} + {q_t}{q_{t}^H} \\
    &= R_{t-1} + q_{t}q_{t}^H
\end{align}

By the Sherman-Morrison formula,

\begin{equation}
R_{t}^{-1} = R_{t-1}^{-1} - \frac{R_{t-1}^{-1}{q_t}{q_{t}^H}R_{t-1}^{-1}}{1 + {q_{t}^H}{R_{t-1}^{-1}}{q_t}}
\end{equation}

Let

\begin{equation}
P_t = R_{t}^{-1} \\
k_t = \frac{R_{t-1}^{-1}{q_t}}{1 + {q_{t}^H}{R_{t-1}^{-1}}{q_t}}
\end{equation}

Then 
\begin{equation}
P_t = P_{t-1} - {k_t}{q_{t}^H}{P_{t-1}}
\end{equation}


## Simple Numerical Examples

Provide some simple python code and examples that emphasize the basic concepts.


## An Engineering Application

Provide a more sophisticated example showing one engineering example of the topic, complete with python code.